In [4]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Cargar el dataset desde el archivo CSV
try:
    data = pd.read_csv('glass.csv')
    # Verificar que la columna 'glass_type' exista
    if 'glass_type' not in data.columns:
        raise ValueError("La columna 'glass_type' no se encuentra en el archivo CSV.")
except FileNotFoundError:
    print("Error: El archivo 'glass.csv' no se encontró. Asegúrate de que el archivo exista en el mismo directorio que el script o proporciona la ruta completa.")
    exit()
except pd.errors.EmptyDataError:
    print("Error: El archivo 'glass.csv' está vacío.")
    exit()
except pd.errors.ParserError:
    print("Error: No se pudo analizar el archivo 'glass.csv'. Verifica el formato del archivo.")
    exit()
except ValueError as e:
    print(f"Error: {e}")
    exit()


# Eliminar filas con NaN en la columna 'glass_type'
data = data.dropna(subset=['glass_type'])

# Definir las características (X) y las etiquetas (y)
X = data.drop('glass_type', axis=1)
y = data['glass_type']

# Escalar las características
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Imprimir el conteo de clases antes de SMOTE
counter = Counter(y_train)
print('Before SMOTE:', counter)

# Aplicar SMOTE
smt = SMOTE(random_state=42)
X_train_sm, y_train_sm = smt.fit_resample(X_train, y_train)

# Imprimir el conteo de clases después de SMOTE
counter = Counter(y_train_sm)
print('After SMOTE:', counter)

Before SMOTE: Counter({2.0: 61, 1.0: 59, 7.0: 20, 3.0: 14, 5.0: 10, 6.0: 6})
After SMOTE: Counter({2.0: 61, 3.0: 61, 1.0: 61, 7.0: 61, 5.0: 61, 6.0: 61})


In [5]:
# 1. Clasificador 1-NN ANTES de SMOTE

# Entrenar el clasificador
knn = KNeighborsClassifier(n_neighbors=1, metric='euclidean')
knn.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred = knn.predict(X_test)

# Evaluar el desempeño (Hold-Out)
accuracy_holdout = accuracy_score(y_test, y_pred)
print("1-NN ANTES de SMOTE (Hold-Out):")
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_holdout:.4f}")

# Validación cruzada (10-Fold)
cv_scores = cross_val_score(knn, X, y, cv=10)
print(f"\n1-NN ANTES de SMOTE (10-Fold Cross-Validation):")
print(f"Accuracy scores: {cv_scores}")
print(f"Mean accuracy: {cv_scores.mean():.4f}")
print(f"Standard deviation: {cv_scores.std():.4f}")


# 2. Aplicar SMOTE y entrenar 1-NN DESPUÉS de SMOTE

smt = SMOTE(random_state=42)
X_train_sm, y_train_sm = smt.fit_resample(X_train, y_train)

# Entrenar el clasificador con los datos sobremuestreados
knn_sm = KNeighborsClassifier(n_neighbors=1, metric='euclidean')
knn_sm.fit(X_train_sm, y_train_sm)

# Predecir en el conjunto de prueba
y_pred_sm = knn_sm.predict(X_test)

# Evaluar el desempeño (Hold-Out)
accuracy_holdout_sm = accuracy_score(y_test, y_pred_sm)
print("\n1-NN DESPUÉS de SMOTE (Hold-Out):")
print(classification_report(y_test, y_pred_sm))
print(f"Accuracy: {accuracy_holdout_sm:.4f}")

# Validación cruzada (10-Fold)
cv_scores_sm = cross_val_score(knn_sm, X, y, cv=10) #Usamos X e y originales para la validacion cruzada
print(f"\n1-NN DESPUÉS de SMOTE (10-Fold Cross-Validation):")
print(f"Accuracy scores: {cv_scores_sm}")
print(f"Mean accuracy: {cv_scores_sm.mean():.4f}")
print(f"Standard deviation: {cv_scores_sm.std():.4f}")

1-NN ANTES de SMOTE (Hold-Out):
              precision    recall  f1-score   support

         1.0       0.91      1.00      0.95        10
         2.0       0.93      0.87      0.90        15
         3.0       1.00      1.00      1.00         3
         5.0       0.60      1.00      0.75         3
         6.0       1.00      0.67      0.80         3
         7.0       1.00      0.89      0.94         9

    accuracy                           0.91        43
   macro avg       0.91      0.90      0.89        43
weighted avg       0.93      0.91      0.91        43

Accuracy: 0.9070

1-NN ANTES de SMOTE (10-Fold Cross-Validation):
Accuracy scores: [0.81818182 0.72727273 0.95454545 0.85714286 0.80952381 0.76190476
 0.95238095 0.71428571 0.80952381 0.85714286]
Mean accuracy: 0.8262
Standard deviation: 0.0783

1-NN DESPUÉS de SMOTE (Hold-Out):
              precision    recall  f1-score   support

         1.0       0.91      1.00      0.95        10
         2.0       0.93      0.87   

c:\Users\Newton\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(
c:\Users\Newton\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(


In [6]:
# Cargar el conjunto de datos Iris
try:
    iris = pd.read_csv('bezdekIris.csv')
    iris = iris[iris['class'].isin(['Iris-setosa', 'versicolor'])]
except FileNotFoundError:
    print("Error: El archivo 'bezdekIris.csv' no se encontró.")
    exit()

# Separar características (X) y etiquetas (y)
X = iris.drop('class', axis=1)
y = iris['class']

# Escalar las características
scaler = StandardScaler()
X = scaler.fit_transform(X) #Escalamos antes de codificar

# Codificar las etiquetas de las clases a valores numéricos (0 y 1)
le = LabelEncoder()
y = le.fit_transform(y) #Codificamos despues de escalar


# Dividir los datos en conjuntos de entrenamiento y prueba (70/30 Hold-Out)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


class Perceptron:
    def __init__(self, learning_rate=0.01, n_iters=1000):
        self.lr = learning_rate
        self.n_iters = n_iters
        self.activation_func = self._unit_step_func
        self.weights = None
        self.bias = None

    def _unit_step_func(self, x):
        return np.where(x >= 0, 1, 0)

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        y_ = np.array([1 if i == 1 else -1 for i in y]) #Ajustamos las etiquetas a 1 y -1

        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                linear_output = np.dot(x_i, self.weights) + self.bias
                y_predicted = self.activation_func(linear_output)

                update = self.lr * (y_[idx] - y_predicted)
                self.weights += update * x_i
                self.bias += update

    def predict(self, X):
        linear_output = np.dot(X, self.weights) + self.bias
        y_predicted = self.activation_func(linear_output)
        return np.where(y_predicted >= 0, 1, 0)


# Entrenar el perceptrón
perceptron = Perceptron(learning_rate=0.01, n_iters=1000)
perceptron.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred = perceptron.predict(X_test)

# Evaluar el desempeño
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print(classification_report(y_test, y_pred))

Accuracy: 0.0000
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      15.0
           1       0.00      0.00      0.00       0.0

    accuracy                           0.00      15.0
   macro avg       0.00      0.00      0.00      15.0
weighted avg       0.00      0.00      0.00      15.0



c:\Users\Newton\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Newton\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Newton\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 